# Experimento 7

Experimento: Probar la estandarización con ventana=2

In [1]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
df = pd.read_csv('../../data/preprocessed/base.csv', sep=',')
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [3]:
dfg = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
dfg.head()

,periodo,product_id,tn
0,201701,20001,934.77222
1,201701,20002,550.15707
2,201701,20003,1063.45835
3,201701,20004,555.91614
4,201701,20005,494.27011


In [4]:
# Función para estandarizar una serie de tiempo
def standardize_series(series):
    mean = series.mean()
    std = series.std()
    standardized_series = (series - mean) / std
    return standardized_series, mean, std

# Función para revertir la estandarización de una serie de tiempo
def reverse_standardization(standardized_series, mean, std):
    reversed_series = standardized_series * std + mean
    return reversed_series

# Estandarizar todas las series de tiempo por product_id
standardized_data = pd.DataFrame()
means = {}
stds = {}
for product_id, group in dfg.groupby('product_id'):
    series = group['tn']
    standardized_series, mean, std = standardize_series(series)
    
    standardized_data = pd.concat([standardized_data, pd.DataFrame({'periodo': group['periodo'], 'product_id': product_id, 'tn_standardized': standardized_series})])
    
    means[product_id] = mean
    stds[product_id] = std

standardized_data.head(5)

# Convertir 'means' y 'stds' en un DataFrame
mean_std_df = pd.DataFrame({'product_id': list(means.keys()),
                            'mean': list(means.values()),
                            'std': list(stds.values())})
mean_std_df.head(1)

,product_id,mean,std
0,20001,1398.344322,298.14546


In [5]:
standardized_data['periodo_dt'] = pd.to_datetime(standardized_data['periodo'].astype(str), format='%Y%m')
standardized_data.rename(columns={'tn_standardized': 'target', 'product_id':'item_id', 'periodo_dt': 'timestamp'}, inplace=True)
standardized_data.drop(columns=['periodo'], inplace=True)
standardized_data.head()

,item_id,target,timestamp
0,20001,-1.554852,2017-01-01
785,20001,-2.013541,2017-02-01
1566,20001,-0.318592,2017-03-01
2352,20001,-1.101419,2017-04-01
3136,20001,0.348343,2017-05-01


##### Levantamos productos a predecir

In [6]:
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
productos_ok.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


##### Filtramos productos a predecir

In [7]:
standardized_data = standardized_data[standardized_data['item_id'].isin(productos_ok['product_id'].unique())]
print(standardized_data.shape)
print(len(standardized_data.item_id.unique()))

(22349, 3)
780


##### Dataset para Autogluon 

In [8]:
data = TimeSeriesDataFrame.from_data_frame(standardized_data,
                                           id_column="item_id",
                                           timestamp_column="timestamp")
data


target
item_id timestamp           
20001   2017-01-01 -1.554852
        2017-02-01 -2.013541
        2017-03-01 -0.318592
        2017-04-01 -1.101419
        2017-05-01  0.348343
...                      ...
21276   2019-08-01 -0.751911
        2019-09-01 -0.616417
        2019-10-01 -0.565291
        2019-11-01 -0.275963
        2019-12-01 -0.837426

[22349 rows x 1 columns]

##### Entrenamiento

In [9]:
predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M").fit(data, num_val_windows = 2)


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\autogluon\timeseries\predictor.py:197: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  std_freq = pd.tseries.frequencies.to_offset(self.freq).freqstr
Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\model_autogluon\AutogluonModels\ag-20250607_180404'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       3.96 GB / 15.89 GB (24.9%)
Disk Space Avail:   411.26 GB / 893.49 GB (46.0%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 

##### Leaderboard

In [36]:
predictor.leaderboard()


,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.034212,15.090804,3.053643,12
1,DirectTabular,-0.034212,15.090804,2805.955686,3
2,RecursiveTabular,-0.176927,1.874368,325.170048,2
3,ChronosZeroShot[bolt_base],-0.190284,10.255714,40.619236,7
4,AutoETS,-0.195550,18.249639,16.638718,6
5,DynamicOptimizedTheta,-0.201990,2.465895,64.111537,5
6,TiDE,-0.231938,1.301187,616.581481,11
7,SeasonalNaive,-0.237611,0.884860,3.809673,1
8,NPTS,-0.288207,1.968401,6.763232,4
9,TemporalFusionTransformer,-0.311265,1.119807,1174.841228,8


##### Feature Importance

TARDA 22 MINUTOS

In [18]:
predictor.feature_importance()

Computing feature importance


,importance,stdev,n,p99_low,p99_high
cat1,0.000136,0.000711,5.0,-0.001329,0.001600
cat2,0.000329,0.000490,5.0,-0.000679,0.001337
cat3,-0.000017,0.000521,5.0,-0.001090,0.001055
brand,-0.000080,0.000570,5.0,-0.001254,0.001095
sku_size,-0.000175,0.001384,5.0,-0.003026,0.002675
crisis,0.003308,0.010835,5.0,-0.019002,0.025619


##### Predecimos

In [10]:
predictions = predictor.predict(data)

data with frequency 'None' has been resampled to frequency 'ME'.


Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


##### Desestandarizamos

In [11]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]].rename(columns = {"mean":"tn"})
df_merged = pd.merge(predictions_v1, mean_std_df.rename(columns = {"product_id":"item_id"}), on='item_id')
def reverse_standardization(standardized_series, mean, std):
    return standardized_series * std + mean
df_merged['tn_original'] = reverse_standardization(df_merged['tn'], df_merged['mean'], df_merged['std'])
df_original = df_merged[['item_id', 'timestamp', 'tn_original']]
df_original.head(5)

,item_id,timestamp,tn_original
0,20001,2020-01-31,1280.499505
1,20001,2020-02-29,1304.867993
2,20002,2020-01-31,1059.326748
3,20002,2020-02-29,1029.338495
4,20003,2020-01-31,798.249515


##### Agrupamos y generamos el df final

In [12]:
predictions_v2 = df_original[df_original.timestamp == "2020-02-29"]
predictions_v2 = predictions_v2.drop(columns = {"timestamp"})
predictions_v2 = predictions_v2.rename(columns = {"item_id":"product_id", "tn_original":"tn"})
predictions_v2.head(5)

,product_id,tn
1,20001,1304.867993
3,20002,1029.338495
5,20003,794.372504
7,20004,559.333180
9,20005,546.713216


##### Generamos archivo

In [13]:
predictions_v2.to_csv("../../outputs/autogluon_con_standarscaler_2ventanas.csv", sep = ",", index = False)